In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Veri setini yükle
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/mnist_train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/mnist_test.csv')

# Veriyi küçült
train_data = train_data.sample(frac=0.1, random_state=1)
test_data = test_data.sample(frac=0.1, random_state=1)

# Bağımsız değişkenleri ve hedef değişkeni ayır
X_train = train_data.drop('label', axis=1).values
y_train = to_categorical(train_data['label'].values)

X_test = test_data.drop('label', axis=1).values
y_test = to_categorical(test_data['label'].values)

# Veriyi normalize et
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Giriş boyutlarını yeniden boyutlandır ve genişlet
X_train_reshaped = X_train.reshape(-1, 28, 28, 1)
X_test_reshaped = X_test.reshape(-1, 28, 28, 1)
X_train_resized = tf.image.resize(X_train_reshaped, (224, 224))
X_test_resized = tf.image.resize(X_test_reshaped, (224, 224))
X_train_resized_rgb = np.stack((X_train_resized[:, :, :, 0],) * 3, axis=-1)
X_test_resized_rgb = np.stack((X_test_resized[:, :, :, 0],) * 3, axis=-1)

# ResNet50 modelini oluştur
base_model = ResNet50(weights=None, include_top=False, input_shape=(224, 224, 3))
model_resnet = models.Sequential()
model_resnet.add(base_model)
model_resnet.add(layers.Flatten())
model_resnet.add(layers.Dense(256, activation='relu'))
model_resnet.add(layers.Dropout(0.5))
model_resnet.add(layers.Dense(10, activation='softmax'))
model_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğit
model_resnet.fit(X_train_resized_rgb, y_train, epochs=5, batch_size=64, validation_split=0.2)

# Modeli değerlendir
y_pred_resnet = model_resnet.predict(X_test_resized_rgb)
y_pred_resnet_classes = np.argmax(y_pred_resnet, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

accuracy_resnet = accuracy_score(y_test_classes, y_pred_resnet_classes)
precision_resnet = precision_score(y_test_classes, y_pred_resnet_classes, average='weighted')
recall_resnet = recall_score(y_test_classes, y_pred_resnet_classes, average='weighted')
f1_resnet = f1_score(y_test_classes, y_pred_resnet_classes, average='weighted')

print(f"Accuracy: {accuracy_resnet}")
print(f"Precision: {precision_resnet}")
print(f"Recall: {recall_resnet}")
print(f"F1 Score: {f1_resnet}")


Epoch 1/5
75/75 [==============================] - 107s 716ms/step - loss: 9.3415 - accuracy: 0.1194 - val_loss: 2.3026 - val_accuracy: 0.1017
Epoch 2/5
75/75 [==============================] - 51s 676ms/step - loss: 2.1174 - accuracy: 0.1690 - val_loss: 2.3969 - val_accuracy: 0.1025
Epoch 3/5
75/75 [==============================] - 50s 662ms/step - loss: 1.9502 - accuracy: 0.2488 - val_loss: 2.5315 - val_accuracy: 0.0875
Epoch 4/5
75/75 [==============================] - 52s 694ms/step - loss: 1.8612 - accuracy: 0.2808 - val_loss: 2.7816 - val_accuracy: 0.0875
Epoch 5/5
32/32 [==============================] - 7s 123ms/step
Accuracy: 0.088
Precision: 0.009428571428571429
Recall: 0.088
F1 Score: 0.01703225806451613


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
